In [15]:
from qiskit.optimization import QuadraticProgram
from qiskit.aqua import aqua_globals
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer

from qiskit.optimization.algorithms import GroverOptimizer
#from qiskit.optimization.problems import QuadraticProgram
MY_API_TOKEN='cc4cc2496aaa2251daf8ff4fc9dfb57d42d03fc3cb4b63253e1ac4e948b413c98c4cfe5bcd630858a18c7442d6d48092b5d08fde332cee022b1a2e20ad5cd35b'
backend=BasicAer.get_backend('qasm_simulator')

In [16]:
def get_best_backend(num_qubits=50, backend='', apiToken=''):
    from qiskit import IBMQ
    
    try:
        IBMQ.enable_account(MY_API_TOKEN)
    except:
        print('Account already enabled')
    print(IBMQ.providers())
    #IBMQ.stored_account()
    #IBMQ.save_account(apiToken, overwrite=True)
    #provider = IBMQ.load_account()
    try:
        provider = IBMQ.get_provider(hub='ibm-q-internal')
    except:
        provider = IBMQ.get_provider(hub='ibm-q')
    print(provider)
    
    if len(backend)>0:
        backend = provider.get_backend(backend)
    else:
        from qiskit.providers.ibmq import least_busy
        large_enough_devices = provider.backends(filters=lambda x: x.configuration().n_qubits > num_qubits 
                                             #and not x.configuration().simulator
                                             )
        backend = least_busy(large_enough_devices)
    return backend

def solve(optimizer, qubo, log=True):
    import logging
    import time
    from qiskit.aqua import set_qiskit_aqua_logging
    
    if qubo.get_num_vars() >10:
        aqua_globals.massive=True
        
    if log==True:
        set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL

    startTime=time.time()
    result=optimizer.solve(qubo)
    runTime=time.time()-startTime
    set_qiskit_aqua_logging(logging.NOTSET)  # OFF

    return result, runTime


Read an LP problem from a file stored in LP Format

In [17]:
#Step 2: Create the QP from the Docplex model
filename='301-Assignment Problem-MILP-B009-133066'
mod = QuadraticProgram()
mod.read_from_lp_file('..\problemFiles\\'+filename+'.lp')
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Assignment Problem

Maximize
 obj: 90 x_0_0 + 80 x_0_1 + 75 x_0_2 + 35 x_1_0 + 85 x_1_1 + 55 x_1_2 + 45 x_2_0
      + 110 x_2_1 + 95 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 <= 1
 c2: x_1_0 + x_1_1 + x_1_2 <= 1
 c3: x_2_0 + x_2_1 + x_2_2 <= 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



To map a problem to the correct input format, the optimization module of Qiskit offers a variety of converters. In this tutorial we’re providing an overview on this functionality. Currently, Qiskit contains the following converters. - InequalityToEquality: converts inequality constraints into equality constraints with additional slack variables. - IntegerToBinary: converts integer variables into binary variables and corresponding coefficients. - LinearEqualityToPenalty: convert equality constraints into additional terms of the object function. - QuadraticProgramToQubo: a wrapper for InequalityToEquality, IntegerToBinary, and LinearEqualityToPenalty for convenience.

### InequalityToEquality

In [18]:
from qiskit.optimization.converters import InequalityToEquality

ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(mod)
print(qp_eq.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Assignment Problem

Maximize
 obj: 90 x_0_0 + 80 x_0_1 + 75 x_0_2 + 35 x_1_0 + 85 x_1_1 + 55 x_1_2 + 45 x_2_0
      + 110 x_2_1 + 95 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 + c1@int_slack = 1
 c2: x_1_0 + x_1_1 + x_1_2 + c2@int_slack = 1
 c3: x_2_0 + x_2_1 + x_2_2 + c3@int_slack = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
       c1@int_slack <= 1
       c2@int_slack <= 1
       c3@int_slack <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2

Generals
 c1@int_slack c2@int_slack c3@int_slack
End



### IntegerToBinary

In [19]:
from qiskit.optimization.converters import IntegerToBinary

#Does not work if there is a quadratic constraint
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Assignment Problem

Maximize
 obj: 90 x_0_0 + 80 x_0_1 + 75 x_0_2 + 35 x_1_0 + 85 x_1_1 + 55 x_1_2 + 45 x_2_0
      + 110 x_2_1 + 95 x_2_2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 + c1@int_slack@0 = 1
 c2: x_1_0 + x_1_1 + x_1_2 + c2@int_slack@0 = 1
 c3: x_2_0 + x_2_1 + x_2_2 + c3@int_slack@0 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= c1@int_slack@0 <= 1
 0 <= c2@int_slack@0 <= 1
 0 <= c3@int_slack@0 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2 c1@int_slack@0
 c2@int_slack@0 c3@int_slack@0
End



### LinearEqualityToPenalty - formation of a QUBO

In [20]:
from qiskit.optimization.converters import LinearEqualityToPenalty

lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC Assignment Problem

Maximize
 obj: 2774 x_0_0 + 2764 x_0_1 + 2759 x_0_2 + 2719 x_1_0 + 2769 x_1_1
      + 2739 x_1_2 + 2729 x_2_0 + 2794 x_2_1 + 2779 x_2_2 + 1342 c1@int_slack@0
      + 1342 c2@int_slack@0 + 1342 c3@int_slack@0 + [ - 2684 x_0_0^2
      - 2684 x_0_0*x_0_1 - 2684 x_0_0*x_0_2 - 2684 x_0_0*x_1_0
      - 2684 x_0_0*x_2_0 - 2684 x_0_0*c1@int_slack@0 - 2684 x_0_1^2
      - 2684 x_0_1*x_0_2 - 2684 x_0_1*x_1_1 - 2684 x_0_1*x_2_1
      - 2684 x_0_1*c1@int_slack@0 - 2684 x_0_2^2 - 2684 x_0_2*x_1_2
      - 2684 x_0_2*x_2_2 - 2684 x_0_2*c1@int_slack@0 - 2684 x_1_0^2
      - 2684 x_1_0*x_1_1 - 2684 x_1_0*x_1_2 - 2684 x_1_0*x_2_0
      - 2684 x_1_0*c2@int_slack@0 - 2684 x_1_1^2 - 2684 x_1_1*x_1_2
      - 2684 x_1_1*x_2_1 - 2684 x_1_1*c2@int_slack@0 - 2684 x_1_2^2
      - 2684 x_1_2*x_2_2 - 2684 x_1_2*c2@int_slack@0 - 2684 x_2_0^2
      - 2684 x_2_0*x_2_1 - 2684 x_2_0*x_2_2 - 2684 x_2_0*c3@int_slack@0
 

# Minimum Eigen Optimizer


Finding the solution to a Quadratic Unconstrained Binary Optimization(QUBO) problem is equivalent in finding the ground state of a corresponding Ising Hamiltonian - applicable to optimization, quantum chemistry and physics.
The binary variables {0,1} are replaced by spin variables with values {−1,+1}, which allows to replace the resulting spin variables by Pauli Z matrices, and thus, an Ising Hamiltonian. 
Details on this mapping -> [1].

Qiskit provides automatic conversion from a suitable QuadraticProgram to an Ising Hamiltonian, which then allows to leverage all the MinimumEigenSolver such as - VQE, - QAOA, or - NumpyMinimumEigensolver(classical exact method).

Qiskit wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called Operator), the call to an MinimumEigensolver as well as the translation of the results back to OptimizationResult in the MinimumEigenOptimizer.

In the following we first illustrate the conversion from a QuadraticProgram to an Operator and then show how to use the MinimumEigenOptimizer with different MinimumEigensolver to solve a given QuadraticProgram. The algorithms in Qiskit automatically try to convert a given problem to the supported problem class if possible, for instance, the MinimumEigenOptimizer will automatically translate integer variables to binary variables or add a linear equality constraints as a quadratic penalty term to the objective. It should be mentioned that Aqua will through a QiskitOptimizationError if conversion of a quadratic program with integer variable is attempted.

The circuit depth of QAOA potentially has to be increased with the problem size, which might be prohibitive for near-term quantum devices. A possible workaround is Recursive QAOA, as introduced in [2]. Qiskit generalizes this concept to the RecursiveMinimumEigenOptimizer, which is introduced at the end of this tutorial.
References

[1] [A. Lucas, Ising formulations of many NP problems, Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, Obstacles to State Preparation and Variational Optimization from Symmetry Protection, arXiv preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)


## Converting a QUBO to an Operator

In [21]:
#Next we translate this QUBO into an Ising operator. 
#This results not only in an Operator but also in a constant offset to be taking into account 
#to shift the resulting value.

op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 5704.0
operator:
SummedOp([
  -961.5 * IIIIIIIIIIIZ,
  -966.5 * IIIIIIIIIIZI,
  -969.0 * IIIIIIIIIZII,
  -989.0 * IIIIIIIIZIII,
  -964.0 * IIIIIIIZIIII,
  -979.0 * IIIIIIZIIIII,
  -984.0 * IIIIIZIIIIII,
  -951.5 * IIIIZIIIIIII,
  -959.0 * IIIZIIIIIIII,
  -671.0 * IIZIIIIIIIII,
  -671.0 * IZIIIIIIIIII,
  -671.0 * ZIIIIIIIIIII,
  335.5 * IIIIIIIIIIZZ,
  335.5 * IIIIIIIIIZIZ,
  335.5 * IIIIIIIIIZZI,
  335.5 * IIIIIIIIZIIZ,
  335.5 * IIIIIIIZIIZI,
  335.5 * IIIIIIIZZIII,
  335.5 * IIIIIIZIIZII,
  335.5 * IIIIIIZIZIII,
  335.5 * IIIIIIZZIIII,
  335.5 * IIIIIZIIIIIZ,
  335.5 * IIIIIZIIZIII,
  335.5 * IIIIZIIIIIZI,
  335.5 * IIIIZIIZIIII,
  335.5 * IIIIZZIIIIII,
  335.5 * IIIZIIIIIZII,
  335.5 * IIIZIIZIIIII,
  335.5 * IIIZIZIIIIII,
  335.5 * IIIZZIIIIIII,
  335.5 * IIZIIIIIIIIZ,
  335.5 * IIZIIIIIIIZI,
  335.5 * IIZIIIIIIZII,
  335.5 * IZIIIIIIZIII,
  335.5 * IZIIIIIZIIII,
  335.5 * IZIIIIZIIIII,
  335.5 * ZIIIIZIIIIII,
  335.5 * ZIIIZIIIIIII,
  335.5 * ZIIZIIIIIIII
])


In [22]:
num_qubits=print(op.num_qubits)

12


In [23]:
#Sometimes a QuadraticProgram might also directly be given in the form of an Operator. 
#For such cases, Qiskit also provides a converter from an Operator back to a QuadraticProgram, 
#which we illustrate in the following.

qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 1432 x_0 - 1422 x_1 - 1417 x_2 - 1377 x_3 - 1427 x_4 - 1397 x_5
      - 1387 x_6 - 1452 x_7 - 1437 x_8 - 671 x_9 - 671 x_10 - 671 x_11 + [
      2684 x_0*x_1 + 2684 x_0*x_2 + 2684 x_0*x_3 + 2684 x_0*x_6 + 2684 x_0*x_9
      + 2684 x_1*x_2 + 2684 x_1*x_4 + 2684 x_1*x_7 + 2684 x_1*x_9 + 2684 x_2*x_5
      + 2684 x_2*x_8 + 2684 x_2*x_9 + 2684 x_3*x_4 + 2684 x_3*x_5 + 2684 x_3*x_6
      + 2684 x_3*x_10 + 2684 x_4*x_5 + 2684 x_4*x_7 + 2684 x_4*x_10
      + 2684 x_5*x_8 + 2684 x_5*x_10 + 2684 x_6*x_7 + 2684 x_6*x_8
      + 2684 x_6*x_11 + 2684 x_7*x_8 + 2684 x_7*x_11 + 2684 x_8*x_11 ]/2 + 4026
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1
 0 <= x_11 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 x_10 x_11
End



In [24]:
qubo=mod

In [25]:
#This converter allows to translate an Operator to a QuadraticProgram and 
#then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, 
#such as the GroverOptimizer.

In [26]:
print(qubo.get_num_vars())

9


In [7]:
backend=get_best_backend(backend='ibmq_manhattan',apiToken=MY_API_TOKEN)

E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>, <AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>]
<AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>


In [8]:
backend=get_best_backend(backend='ibmq_montreal',apiToken=MY_API_TOKEN)

E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>, <AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>]
<AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>


## Solving a QUBO with the MinimumEigenOptimizer

In [27]:
#We start by initializing the MinimumEigensolver we want to use.
#backend=BasicAer.get_backend('qasm_simulator')
aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(backend, 
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed,
                                   optimization_level=3,
                                  skip_qobj_validation=False)

vqe_core = VQE(quantum_instance=quantum_instance, max_evals_grouped=101)

qaoa_core = QAOA(quantum_instance=quantum_instance, max_evals_grouped=101)
#qaoa_core = QAOA(quantum_instance=backend)

# Create a Grover Optimizer - it will terminate after there have been 10 iterations of GAS without progress (i.e. the value of y does not change). 
grover_core = GroverOptimizer(qubo.get_num_vars(), num_iterations=5, quantum_instance=backend) #, max_evals_grouped=101)

exact_core = NumPyMinimumEigensolver()

#Then, we use the MinimumEigensolver to create MinimumEigenOptimizer.
qaoa = MinimumEigenOptimizer(qaoa_core)    # using QAOA
vqe = MinimumEigenOptimizer(vqe_core)      # using VQE
#grover = MinimumEigenOptimizer(grover_core)# using Grover's
exact = MinimumEigenOptimizer(exact_core)  # using the exact classical numpy minimum eigen solver

2021-01-01 20:16:58,549:qiskit.aqua.quantum_instance:INFO: 
Qiskit Terra version: 0.16.1
Backend: 'qasm_simulator (BasicAer)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'cx', 'id', 'unitary'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': 3}
RunConfig(max_credits=10, seed_simulator=10598, shots=1024)
{'timeout': None}
{}
{}
Measurement mitigation: None
2021-01-01 20:16:59,975:qiskit.aqua.algorithms.minimum_eigen_solvers.vqe:INFO: 
==================== Setting of VQE ============================
Algorithm: VQE
-- max_evals_grouped: 101
-- circuit_sampler: <qiskit.aqua.operators.converters.circuit_sampler.CircuitSampler object at 0x000001FAEBB62448>
-- expectation: None
-- user_valid_expectation: False
-- include_custom: False
-- expect_op: None
-- operator: None
-- quantum_instance: 
Qiskit Terra version: 0.16.1
Backend: 'qasm_simulator (BasicAer)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'cx', 'id', 'u

In [28]:
#We first use the MinimumEigenOptimizer based on the classical exact 
#NumPyMinimumEigensolver to get the optimal benchmark solution for this small example.
exact_result, runTime = solve(exact, qubo)
print(exact_result)
print('Time Taken:', runTime)

2021-01-01 20:17:14,727:qiskit.aqua.algorithms.eigen_solvers.numpy_eigen_solver:DEBUG: NumPyEigensolver _run result:
{   'eigvals': array([-5974.+0.j]),
    'eigvecs': array([[0., 0., 0., ..., 0., 0., 0.]]),
    'energies': array([-5974.]),
    'energy': -5974.0,
    'wavefunction': array([[0., 0., 0., ..., 0., 0., 0.]])}
2021-01-01 20:17:15,166:qiskit.aqua.algorithms.eigen_solvers.numpy_eigen_solver:DEBUG: EigensolverResult dict:
{   'eigenstates': ListOp([VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False)], coeff=1.0, abelian=False),
    'eigenvalues': array([-5974.+0.j])}
2021-01-01 20:17:15,182:qiskit.aqua.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver:DEBUG: NumPyMinimumEigensolver dict:
{   'eigenstate': VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, 

optimal function value: 270.0
optimal value: [1. 0. 0. 0. 1. 0. 0. 0. 1.]
status: SUCCESS
Time Taken: 1.1093721389770508


In [ ]:
def log_results(result,solver, runTime, env='Classical'):
    #pe	variables	Optimal Function Value	Result	RunTime (seconds)	Total Quantum RunTime (second)	Qubits	Operations	Environment	Solver	Solution
    entry=filename+','+'MILP'+','+str(qubo.get_num_vars())+','+'%.4f'%result.fval+','
    for x in result.x:
        entry = entry+'%.0f'%(x)+'  '
    
    solverText=solver.__class__.__name__+'('+solver.min_eigen_solver.__class__.__name__+')'
    qubits=0
    operations=0
    if env!='Classical':
        qubits = solver.min_eigen_solver.get_optimal_circuit().num_qubits
        operations=str(solver.min_eigen_solver.get_optimal_circuit().size())+' Depth - '+str(solver.min_eigen_solver.get_optimal_circuit().depth())
        
    entry=entry+','+'%.2f'%(runTime)+',,'+str(qubits)+','+str(operations)+','+env+','+ \
        solverText+','+str(result.status).split('.')[1]+'\n'
    print(entry)
    #f=open('..\\stats.csv','a')
    #f.write(entry)
    #f.close()

#log_results(exact_result, exact, runTime)

In [41]:
#Next we apply the MinimumEigenOptimizer based on QAOA to the same problem.
qaoa_result, qrunTime = solve(qaoa, qubo)
#qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

2021-01-01 20:29:50,113:qiskit.aqua.algorithms.vq_algorithm:INFO: Starting optimizer.
bounds=[(0, 3.141592653589793), (0, 6.283185307179586)]
initial point=[0, 0]
2021-01-01 20:29:53,622:qiskit.aqua.quantum_instance:DEBUG: ==== Before transpiler ====
2021-01-01 20:29:53,638:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 12 qubits, 12 classical bits and 129 operations with depth 43
op_counts: OrderedDict([('cx', 54), ('rz', 39), ('h', 12), ('rx', 12), ('measure', 12)])

2021-01-01 20:29:53,638:qiskit.aqua.quantum_instance:DEBUG: ====  After transpiler ====
2021-01-01 20:29:53,638:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 12 qubits, 12 classical bits and 129 operations with depth 43
op_counts: OrderedDict([('cx', 54), ('u1', 39), ('u2', 12), ('u3', 12), ('measure', 12)])

2021-01-01 20:29:53,685:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter binding 46.87262 (ms)
2021-01-01 20:29:58,867:qiskit.aqua.algorithms.min

optimal function value: 205.0
optimal value: [0. 0. 1. 0. 1. 0. 1. 0. 0.]
status: SUCCESS


In [53]:
print('Time Taken:', qrunTime)
log_results(qaoa_result, qaoa, qrunTime, env=backend.name())

Time Taken: 19.148793935775757
301-Assignment Problem-MILP-B009-133066,MILP,9,205.0000,0  0  1  0  1  0  1  0  0  ,19.15,,12,117 Depth - 42,qasm_simulator,MinimumEigenOptimizer(QAOA),SUCCESS



In [54]:
#Next we apply the MinimumEigenOptimizer based on VQE to the same problem.
vqe_result, vqerunTime = solve(vqe, qubo)
print(vqe_result)

2021-01-01 20:42:17,589:qiskit.aqua.algorithms.vq_algorithm:INFO: Starting optimizer.
bounds=[(-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.

optimal function value: 205.0
optimal value: [0. 0. 1. 0. 1. 0. 1. 0. 0.]
status: SUCCESS


In [55]:
print(vqerunTime)
log_results(vqe_result, vqe, vqerunTime, env=backend.name())

89.43233919143677
301-Assignment Problem-MILP-B009-133066,MILP,9,205.0000,0  0  1  0  1  0  1  0  0  ,89.43,,12,246 Depth - 49,qasm_simulator,MinimumEigenOptimizer(VQE),SUCCESS



## RecursiveMinimumEigenOptimizer

In [56]:
#The RecursiveMinimumEigenOptimizer takes a MinimumEigenOptimizer as input and 
#applies the recursive optimization scheme to reduce the size of the problem one variable at a time. 
#Once the size of the generated intermediate problem is below a given threshold (min_num_vars), 
#the RecursiveMinimumEigenOptimizer uses another solver (min_num_vars_optimizer), 
#e.g., an exact classical solver such as CPLEX or the MinimumEigenOptimizer based on the NumPyMinimumEigensolver.

#In the following, we show how to use the RecursiveMinimumEigenOptimizer 
#using the two MinimumEigenOptimizer introduced before.

#First, we construct the RecursiveMinimumEigenOptimizer 
#such that it reduces the problem size from 3 variables to 1 variable and 
#then uses the exact solver for the last variable. 
#Then we call solve to optimize the considered problem.

rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=5, min_num_vars_optimizer=exact)

rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)



constraint c3 is infeasible due to substitution
constraint c4 is infeasible due to substitution


optimal function value: 300.0
optimal value: [1. 0. 0. 0. 0. 1. 1. 1. 0.]
status: INFEASIBLE


In [ ]:
# Solve using Grover's Optimizer
grover_result, groverRunTime = solve(grover_core, qubo)
print(grover_result)

In [1]:
print(groverRunTime)
log_results(grover_result, grover_core, groverRunTime, env=backend.name())

NameError: name 'groverRunTime' is not defined

In [ ]:
import logging
import time
from qiskit.aqua import set_qiskit_aqua_logging

if qubo.get_num_vars() >10:
    aqua_globals.massive=True
set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL

startTime=time.time()
result = grover_core.solve(qubo)
runTime=time.time()-startTime
#set_qiskit_aqua_logging(logging.NOTSET)  # OFF
print("x={}".format(result.x))
print("fval={}".format(result.fval))
print('Runtime: ', runTime)

2020-12-30 13:42:00,214:qiskit.aqua.utils.run_circuits:INFO: Backend status: <qiskit.providers.models.backendstatus.BackendStatus object at 0x000001C01C24BEC8>
2020-12-30 13:42:00,215:qiskit.aqua.utils.run_circuits:INFO: There are 1 jobs are submitted.
2020-12-30 13:42:00,216:qiskit.aqua.utils.run_circuits:INFO: All job ids:
['5fec3649e60331001b30981f']
2020-12-30 13:42:00,216:qiskit.aqua.utils.run_circuits:INFO: Running 0-th qobj, job id: 5fec3649e60331001b30981f
2020-12-30 13:42:01,173:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f, status: JobStatus.VALIDATING
2020-12-30 13:42:07,251:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:12,759:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:18,834:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:24,876:qiskit.aqua.utils.run_circuit